In [1]:
import numpy as np
import pandas as pd
import requests 
import json
import re

from tqdm import tqdm

In [2]:
# trains=pd.read_csv('서울특별시_부동산_실거래가_정보_2020년.csv',encoding='cp949',chunksize=40000)
# train=pd.concat(trains)

# train=train[['자치구명','법정동명','대지권면적','건물면적','층정보','건물주용도','물건금액','건축년도','건물명']]
# train=train.loc[(train['건물주용도']=='아파트')&(train['자치구명']=='강남구')]

# train.to_csv('2020강남아파트실거래가.csv',index=False)

In [3]:
# house=pd.read_csv('2020강남아파트실거래가.csv')

# house['주소']=house.apply(lambda row:row.자치구명+' '+row.법정동명+' '+row.건물명,axis=1)

# def getLatLng(addr):
#     url='https://dapi.kakao.com/v2/local/search/keyword.json?query=' + addr
#     headers={"Authorization": "KakaoAK 9e25ea8cb470e4f867926e9db394dde6"}
#     res=requests.get(url, headers=headers)
#     res.raise_for_status() 
#     match=res.json()['documents'][0]
#     return float(match['y']), float(match['x'])

# for idx,addr in enumerate(house['주소']):
#     try:
#         y,x=getLatLng(addr)
#         house.loc[idx,'y']=y
#         house.loc[idx,'x']=x
#     except:
#         continue
        
# data=house[house['y'].isnull()]['주소']

# for idx,addr in zip(data.index,data):
#     addr=re.sub(r'\(.+\)','',addr)
#     try:
#         y,x=getLatLng(addr)
#         house.loc[idx,'y']=y
#         house.loc[idx,'x']=x
#     except:
#         continue
        
# house['주소2']=house.apply(lambda row:row.자치구명+' '+row.건물명,axis=1)
# house['주소3']=house.apply(lambda row:row.법정동명+' '+row.건물명,axis=1)

# data=house[house['y'].isnull()]['주소2']

# for idx,addr in zip(data.index,data):
#     addr=re.sub(r'\(.+\)','',addr)
#     try:
#         y,x=getLatLng(addr)
#         house.loc[idx,'y']=y
#         house.loc[idx,'x']=x
#     except:
#         continue
        
# data=house[house['y'].isnull()]['주소3']

# for idx,addr in zip(data.index,data):
#     addr=re.sub(r'\(.+\)','',addr)
#     try:
#         y,x=getLatLng(addr)
#         house.loc[idx,'y']=y
#         house.loc[idx,'x']=x
#     except:
#         continue
        
# # house.to_csv('house_addr.csv',index=False)
# house=pd.read_csv('house_addr.csv')

In [4]:
# house=house.dropna(subset=['y'])
# house=house.drop(['대지권면적'],axis=1)
# house=house.reset_index(drop=True)

# ## 학교

# def getCat(x,y,radius=500):
#     url=f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code=SC4&x={x}&y={y}&radius={radius}&sort=distance'
#     headers={"Authorization": "KakaoAK 9e25ea8cb470e4f867926e9db394dde6"}
#     res=requests.get(url, headers=headers)
#     res.raise_for_status() 
#     cnt=res.json()['meta']['total_count']
#     return cnt

# for idx,(x,y) in enumerate(zip(house['x'],house['y'])):
#     cnt=getCat(x,y)
#     house.loc[idx,'근처학교수']=cnt

# house.to_csv('house(add_school).csv',index=False)

In [5]:
# ## 500m이내 복지시설 수 

# house=pd.read_csv('house(add_school).csv')

# ## 보편화

# def getCat(cat,x,y,radius=500):
#     url=f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code={cat}&x={x}&y={y}&radius={radius}&sort=distance'
#     headers={"Authorization":"KakaoAK 9e25ea8cb470e4f867926e9db394dde6"}
#     res=requests.get(url,headers=headers)
#     res.raise_for_status() 
#     cnt=res.json()['meta']['total_count']
#     return cnt

# cat_code={'MT1':'대형마트','CS2':'편의점','PS3':'어린이집,유치원','SC4':'학교','AC5':'학원','PK6':'주차장','OL7':'주유소,충전소',
#           'SW8':'지하철역','BK9':'은행','CT1':'문화시설','AG2':'중개업소','PO3':'공공기관','AT4':'관광명소','AD5':'숙박','FD6':'음식점',
#           'CE7':'카페','HP8':'병원','PM9':'약국'}

# for cat in cat_code.keys():
#     if cat=='SC4':
#         continue
#     for idx,(x,y) in enumerate(zip(house['x'],house['y'])):
#         cnt=getCat(cat,x,y)
#         house.loc[idx,f'근처{cat_code[cat]}수']=cnt

# # house.to_csv('house(kakao).csv',index=False)

In [54]:
# 작년 시세 (경제요인)

house=pd.read_csv('house(kakao).csv')

trains=pd.read_csv('서울특별시_부동산_실거래가_정보_2019년.csv',encoding='cp949',chunksize=40000)
train=pd.concat(trains)

train=train[['자치구명','법정동명','대지권면적','건물면적','층정보','건물주용도','물건금액','건축년도','건물명']]
train=train.loc[(train['건물주용도']=='아파트')&(train['자치구명']=='강남구')]

house=pd.merge(house,train[['자치구명','법정동명','건물명','물건금액','건물면적','층정보','건축년도']],
               on=['자치구명','법정동명','건물명','건물면적','층정보','건축년도'],how='left').dropna()

house=house.rename(columns={'물건금액_x':'2020년물건금액','물건금액_y':'2019년물건금액'})

# 재건축 특성
house['건축년도제곱']=np.power(house['건축년도'],2)

## 중복값 제거
house=house.drop_duplicates(['자치구명','법정동명','건물명','건물면적','층정보','건축년도']).reset_index(drop=True)
# house.to_excel('아파트실거래가.xlsx')

C:\Users\ckdck\miniconda3\envs\eda\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [56]:
# house.to_excel('아파트실거래가.xlsx')

In [ ]:
import requests

## snip : 주소를 통한 위,경도 가져오기

house['주소']=house.apply(lambda row:row.자치구명+' '+row.건물명,axis=1)

def getLatLng(addr):
    url='https://dapi.kakao.com/v2/local/search/keyword.json?query=' + addr
    headers={"Authorization": "KakaoAK ********************************"}
    res=requests.get(url,headers=headers)
    res.raise_for_status() 
    match=res.json()['documents'][0]
    return float(match['y']), float(match['x'])

for idx,addr in enumerate(house['주소']):
    try:
        y,x=getLatLng(addr)
        house.loc[idx,'y']=y
        house.loc[idx,'x']=x
    except:
        continue
        
## end snip

In [ ]:
## snip : 위.경도 기준 500m이내 주변 시설 수 

def getCat(cat,x,y,radius=500):
    url=f'https://dapi.kakao.com/v2/local/search/category.json?category_group_code={cat}&x={x}&y={y}&radius={radius}&sort=distance'
    headers={"Authorization":"KakaoAK ********************************"}
    res=requests.get(url,headers=headers)
    res.raise_for_status() 
    cnt=res.json()['meta']['total_count']
    return cnt

cat_code={'MT1':'대형마트','CS2':'편의점','PS3':'어린이집,유치원','SC4':'학교','AC5':'학원','PK6':'주차장','OL7':'주유소,충전소',
          'SW8':'지하철역','BK9':'은행','CT1':'문화시설','AG2':'중개업소','PO3':'공공기관','AT4':'관광명소','AD5':'숙박','FD6':'음식점',
          'CE7':'카페','HP8':'병원','PM9':'약국'}

## enc snip